In [ ]:
import json
import copy
import os
import numpy as np
from shutil import copyfile
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw

from skimage.draw import polygon
from skimage.measure import regionprops, label
from skimage.morphology import medial_axis

In [ ]:
def extract_name(feature):
    if 'name' in feature['properties']:
        return feature['properties']['name']
    elif 'wikipedia' in feature['properties']:
        return feature['properties']['wikipedia']
    else:
        return None
    
    
def extract_coordinates(feature):
    x_coordinates = []
    y_coordinates = []
    for point in feature['geometry']['coordinates'][0]:
        x_coordinates.append(point[0])
        y_coordinates.append(point[1])
                
    return x_coordinates, y_coordinates


def binary_image(im_size, x_coordinates, y_coordinates):
    x_coordinates = np.array(x_coordinates)
    y_coordinates = np.array(y_coordinates)

    x_coordinates -= min(x_coordinates)
    y_coordinates -= min(y_coordinates)


    x_max = max(x_coordinates)
    y_max = max(y_coordinates)

    if x_max >= y_max:
        ratio = im_size/x_max

    else:
        ratio = im_size/y_max

    x_coordinates *= ratio
    y_coordinates *= ratio

    img = np.zeros((im_size, im_size), dtype=np.uint8)

    x, y = polygon(x_coordinates, y_coordinates)
    img[x, y] = 1
    
    return img

def binary_image_center(im_size, x_coordinates, y_coordinates, y_centroid):
    # In that space, y and x axis are inverted
    x_centroid = y_centroid
    
    x_coordinates = np.array(x_coordinates)
    y_coordinates = np.array(y_coordinates)

    x_coordinates -= min(x_coordinates)
    y_coordinates -= min(y_coordinates)

    x_max = max(x_coordinates)
    y_max = max(y_coordinates)

    ratio = im_size / y_max

    x_coordinates *= ratio
    y_coordinates *= ratio

    x_coordinates = np.array([min(im_size - 1, max(0, x + im_size / 2 - x_centroid)) for x in x_coordinates])
    y_coordinates = np.array([min(im_size - 1, max(0, y)) for y in y_coordinates])
    
    img = np.zeros((im_size, im_size), dtype=np.uint8)

    x, y = polygon(x_coordinates, y_coordinates)
    img[x, y] = 1
    
    return img


def unique_orientation_coords(bin_image, x_coords, y_coords):
    props = regionprops(bin_image)

    angle = -props[0].orientation
    cos = np.cos(angle)
    sin = np.sin(angle)

    x_coords_rot = []
    y_coords_rot = []

    for i in range(len(x_coords)):
        x = x_coords[i]
        y = y_coords[i]

        x_coords_rot.append(x * cos - y * sin)
        y_coords_rot.append(y * cos + x * sin)

        
    return x_coords_rot, y_coords_rot, angle


def most_frequent_colour(image):

    w, h = image.size
    pixels = image.getcolors(w * h)

    most_frequent_pixel = pixels[0]

    for count, colour in pixels:
        if count > most_frequent_pixel[0] and colour != (0, 0, 0):
            most_frequent_pixel = (count, colour)

    return most_frequent_pixel[1]

In [ ]:
with open('geojson/lacs_suisse.geojson') as f:
    lakes_geo = json.load(f)

#### Conversion to format accepted by Leonardo's code

In [ ]:
lakes_leonardo = []

for feature in lakes_geo['features']:
    if feature['geometry']['type'] == 'Polygon':
        lake = {
            'id': feature['properties']['@id'].replace('/', '_'),
            'vertices': feature['geometry']['coordinates'][0],
            'name': extract_name(feature),
        }

        lakes_leonardo.append(lake)

In [ ]:
with open('json/lakes_leo_suisse.json', 'w') as f:
    
    f.write(json.dumps(lakes_leonardo))

# Now use leo's script to download + crop images

...

In [ ]:
cropped_path = 'leonardo-git/maps/images/processed'
raw_path = 'leonardo-git/maps/images/raw'
collection_path = 'lake_collection'
json_path = 'json'

## Copy files in collection folder 

In [ ]:
lakes_infos = {}

for lake in lakes_leonardo:
    lakes_infos[lake['id']] = {'name': lake['name']}

In [ ]:
for lake_id in lakes_infos.keys():
    
    # Raw image
    raw_src = os.path.join(raw_path, lake_id + '.png')
    raw_dst = os.path.join(collection_path, lake_id + '_raw.png')
    
    # Cropped image
    cropped_src = os.path.join(cropped_path, lake_id + '.png')
    cropped_dst = os.path.join(collection_path, lake_id + '_cropped.png')
    
    # Mask image
    mask_src = os.path.join(cropped_path, lake_id + '_mask.png')
    mask_dst = os.path.join(collection_path, lake_id + '_mask.png')
    
    # Copy files
    copyfile(raw_src, raw_dst)
    copyfile(cropped_src, cropped_dst)
    copyfile(mask_src, mask_dst)

## Binary image, angle, skeleton

In [ ]:
def centroid_y(bin_img, im_size):
    y_counts = np.sum(bin_img, axis=1)
    indices = np.arange(im_size)

    return sum(y_counts * indices) / sum(y_counts)

In [ ]:
im_size = 50

binary_images = {}
distances_images_orig = {}
distances_images = {}

for feature in lakes_geo['features']: #TODO ALL
    if feature['geometry']['type'] == 'Polygon':
        try:
            id_ = feature['properties']['@id'].replace('/', '_')
            coordinates = extract_coordinates(feature)
            bin_image = binary_image(im_size, *coordinates)
            
            skeleton_orig, distance_orig = medial_axis(bin_image, return_distance=True)

            x_coords_rot, y_coords_rot, angle = unique_orientation_coords(bin_image, coordinates[0], coordinates[1])
            bin_image_rot = binary_image(im_size, x_coords_rot, y_coords_rot)

            centroid = centroid_y(bin_image_rot, im_size)
            bin_image_center = binary_image_center(im_size, x_coords_rot, y_coords_rot, centroid)

            skeleton, distance = medial_axis(bin_image_center, return_distance=True)

            cropped = Image.open(os.path.join(collection_path, id_ + '_cropped.png'))
            most_freq_color = most_frequent_colour(cropped)

            lakes_infos[id_]['angle'] = angle
            lakes_infos[id_]['most_freq_color'] = most_freq_color
            lakes_infos[id_]['first_coordinate'] = (coordinates[1][0], coordinates[0][0])

            binary_images[id_] = bin_image.tolist()
            distances_images_orig[id_] = distance_orig.tolist()
            distances_images[id_] = distance.tolist()
            
        except Exception as e:
            del lakes_infos[id_]
            print(e)
            print('Skipped ' + id_)#TODO remove from object

In [ ]:
with open(os.path.join(json_path, 'lakes_infos.json'), 'w') as f:
    f.write(json.dumps(lakes_infos))
    
with open(os.path.join(json_path, 'binary_images.json'), 'w') as f:
    f.write(json.dumps(binary_images))
    
with open(os.path.join(json_path, 'distances_images.json'), 'w') as f:
    f.write(json.dumps(distances_images))
    
with open(os.path.join(json_path, 'distances_images_orig.json'), 'w') as f:
    f.write(json.dumps(distances_images_orig))

In [ ]:
#with open(os.path.join(collection_path, 'lakes_infos.json'), 'r') as f:
#    lakes_infos = json.load(f)